In [ ]:
"""Example for plotting gradient data"""
import os.path as op
from glob import glob
import pickle

import numpy as np
import matplotlib.image as mpimg
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
from nibabel import GiftiImage
from nibabel.gifti import GiftiDataArray
from surfplot.utils import add_fslr_medial_wall
from segmentation import KDESegmentation, KMeansSegmentation, PCTLSegmentation
import nibabel as nib
import numpy as np

from utils import plot_gradient

In [ ]:
data_dir = "../data"
figures_dir = op.abspath("../figures")

In [ ]:
gradients = np.load("../results/gradient/gradients.npy")

template_dir = "../data/templates"
subcortical_fn = op.join(template_dir, "rois-subcortical_mni152_mask.nii.gz")
subcort_img = nib.load(subcortical_fn)

full_vertices = 64984
hemi_vertices = full_vertices // 2

subcort_dat = subcort_img.get_fdata()
subcort_mask = subcort_dat != 0
n_subcort_vox = np.where(subcort_mask)[0].shape[0]

n_gradients = gradients.shape[1]
grad_lst = []
for i in range(n_gradients):
    cort_grads = gradients[: gradients.shape[0] - n_subcort_vox, i]
    grad_lst.append(cort_grads)

gradient = grad_lst[0]

# Visualize segmentation

## Percentile

In [ ]:
percent_grad_seg_path = "../results/segmentation/PCT_gradient-maps"
percent_grad_out_path = "../figures/Fig/segmentation/pct"
percent_grad_seg_lh_fnames = sorted(glob(op.join(percent_grad_seg_path, "*PCT*_desc-C*-L_feature.func.gii")))
percent_grad_seg_rh_fnames = sorted(glob(op.join(percent_grad_seg_path, "*PCT*_desc-C*-R_feature.func.gii")))
percent_grad_seg_fnames = zip(percent_grad_seg_lh_fnames, percent_grad_seg_rh_fnames)

plot_gradient(data_dir, percent_grad_seg_fnames, cmap="YlOrRd", color_range=(0,1), title=False, out_dir=percent_grad_out_path)

## KMeans

In [ ]:
kmeans_grad_seg_path = "../results/segmentation/KMeans_gradient-maps"
kmeans_grad_out_path = "../figures/Fig/segmentation/kmeans"
kmeans_grad_seg_lh_fnames = sorted(glob(op.join(kmeans_grad_seg_path, "*KMeans*_desc-C*-L_feature.func.gii")))
kmeans_grad_seg_rh_fnames = sorted(glob(op.join(kmeans_grad_seg_path, "*KMeans*_desc-C*-R_feature.func.gii")))
kmeans_grad_seg_fnames = zip(kmeans_grad_seg_lh_fnames, kmeans_grad_seg_rh_fnames)

plot_gradient(data_dir, kmeans_grad_seg_fnames, cmap="YlOrRd", color_range=(0,1), title=False, out_dir=kmeans_grad_out_path)

## KDE

In [ ]:
kde_grad_seg_path = "../results/segmentation/KDE_gradient-maps"
kde_grad_out_path = "../figures/Fig/segmentation/kde"
kde_grad_seg_lh_fnames = sorted(glob(op.join(kde_grad_seg_path, "*KDE*_desc-C*-L_feature.func.gii")))
kde_grad_seg_rh_fnames = sorted(glob(op.join(kde_grad_seg_path, "*KDE*_desc-C*-R_feature.func.gii")))
print(kde_grad_seg_lh_fnames)
print(kde_grad_seg_rh_fnames)
kde_grad_seg_fnames = zip(kde_grad_seg_lh_fnames, kde_grad_seg_rh_fnames)

plot_gradient(data_dir, kde_grad_seg_fnames, cmap="YlOrRd", color_range=(0,1), title=False, out_dir=kde_grad_out_path)

In [ ]:
import gc

from neuromaps.datasets import fetch_fslr
from surfplot import Plot
from surfplot.utils import threshold
import matplotlib.pyplot as plt


def plot_surf_maps(lh_grad, rh_grad, color_range, cmap, dpi, data_dir, out_filename):
    neuromaps_dir = op.join(data_dir, "neuromaps")

    surfaces = fetch_fslr(density="32k", data_dir=neuromaps_dir)
    lh, rh = surfaces["inflated"]
    sulc_lh, sulc_rh = surfaces["sulc"]

    p = Plot(lh, views="lateral")
    p.add_layer({"left": sulc_lh}, cmap="binary_r", cbar=False)
    p.add_layer({"left": lh_grad}, cmap=cmap, cbar=False, color_range=color_range,)
    fig = p.build()

    fig.savefig(out_filename, bbox_inches="tight", dpi=dpi, transparent=True)
    fig = None
    plt.close()
    gc.collect()
    plt.clf()

In [ ]:
segmentations = [3, 17, 32]
seg_sols = [[0, 1, 2], [0, 8, 16], [0, 15, 31]]
methods = ["PCT", "KMeans", "KDE"]
grad_out_path = "../figures/Fig/segmentation"

for seg_i, segmentation in enumerate(segmentations):
    print(f"Segmentation: {segmentation}")
    for seg_sol in seg_sols[seg_i]:
        for method in methods:
            grad_seg_path = f"../results/segmentation/{method}_gradient-maps"
            grad_seg_lh_fnames = sorted(glob(op.join(grad_seg_path, f"*{method}{segmentation:02d}_desc-C{seg_sol:02d}_*-L_feature.func.gii")))
            grad_seg_rh_fnames = sorted(glob(op.join(grad_seg_path, f"*{method}{segmentation:02d}_desc-C{seg_sol:02d}_*-R_feature.func.gii")))
            print(grad_seg_lh_fnames)
            print(grad_seg_rh_fnames)
            grad_seg_fnames = zip(grad_seg_lh_fnames, grad_seg_rh_fnames)

            # plot_gradient(data_dir, grad_seg_fnames, cmap="YlOrRd", color_range=(0,1), views="lateral", title=False, out_dir=grad_out_path)
            for lh_grad, rh_grad in grad_seg_fnames:
                out_filename = op.join(grad_out_path, f"{method}{segmentation:02d}_C{seg_sol:02d}.tiff")
                plot_surf_maps(lh_grad, rh_grad, (0,1), "YlOrRd", 100, data_dir, out_filename)


In [ ]:
n_cols = 3
n_rows = 5
w = 7.5
h = 9.5

img_lbs = ["PCT", "KMeans", "KDE"]
step = 0
row = 0
for segment_size in range(2, 33):
    pct_files = sorted(glob(op.join(figures_dir, "Fig", "segmentation", "pct", f"*{segment_size:02d}-*.tiff")))
    kms_files = sorted(glob(op.join(figures_dir, "Fig", "segmentation", "kmeans", f"*{segment_size:02d}-*.tiff")))
    kde_files = sorted(glob(op.join(figures_dir, "Fig", "segmentation", "kde", f"*{segment_size:02d}-*.tiff")))
    for segment_id, (pct_file, kms_file, kde_file) in enumerate(zip(pct_files, kms_files, kde_files), start=1):
        add_title = False
        if step % 5 == 0:
            add_title = True
        
        step += 1
        
        if row == 0:
            fig = plt.figure(figsize=(w, h))
            fig.subplots_adjust(
                left=None, bottom=None, right=None, top=None, wspace=0.9, hspace=None
            )
            gs = GridSpec(n_rows, n_cols, figure=fig)

        for img_i, img_file in enumerate([pct_file, kms_file, kde_file]):
            img = mpimg.imread(img_file)
            ax = fig.add_subplot(gs[row, img_i], aspect="equal")
            ax.imshow(img)

            if img_i == 0:
                ax.set_xticks([])
                ax.set_yticks([])
                if add_title:
                    ax.set_ylabel(f"Segment\nSolution\n\n\n\n{segment_size:02d}-{segment_id:02d}", rotation=0, labelpad=35, fontsize=12)
                else:
                    ax.set_ylabel(f"\n{segment_size:02d}-{segment_id:02d}", rotation=0, labelpad=35, fontsize=12)
                plt.setp(ax.spines.values(), color=None)
            else:
                ax.set_axis_off()
            
            if add_title:
                ax.set_title(img_lbs[img_i], fontsize=14)
        
        if row == 4:
            row = 0
            fig.tight_layout(pad=0.1, w_pad=0.6, h_pad=0.1)
            # plt.subplots_adjust(top=0.95)
            fig.savefig(op.join(figures_dir, "Fig", "segmentation", f"{segment_size:02d}-{segment_id:02d}_gradient-maps.eps"), bbox_inches="tight", dpi=500)
            print(f"\includegraphics[scale=1]{{{segment_size:02d}-{segment_id:02d}_gradient-maps.eps}}\n")
            plt.close()
            plt.clf()
        else:
            row += 1
        # suplabel = fig.supylabel(f"Segmt. Sol. - ID: {segment_size:02d} - {segment_id:02d}", fontsize=10)

In [ ]:
with open(op.join("../results/segmentation", "new_PCT_results.pkl"), "rb") as results_file:
    pct_dict = pickle.load(results_file)
with open(op.join("../results/segmentation", "new_KMeans_results.pkl"), "rb") as results_file:
    kmeans_dict = pickle.load(results_file)
with open(op.join("../results/segmentation", "new_KDE_results.pkl"), "rb") as results_file:
    kde_dict = pickle.load(results_file)

dict_list = [pct_dict, kmeans_dict, kde_dict]
label_list = ["PCT", "KMeans", "KDE"]

In [ ]:
from sklearn import metrics
import seaborn as sns
import pandas as pd

In [ ]:
methods_lst = []
seg_sol_lst = []
metrics_lst = []
for seg in range(31):
    for i in range(len(dict_list)):
        for j in range(i):
            nmi = metrics.normalized_mutual_info_score(
                dict_list[i]["labels"][seg],
                dict_list[j]["labels"][seg],
            )
            methods_lst.append(f"{label_list[i]} ~ {label_list[j]}")
            seg_sol_lst.append(seg+2)
            metrics_lst.append(nmi)


In [ ]:
data_df = pd.DataFrame({"Method": methods_lst, "Segmentation": seg_sol_lst, "NMI": metrics_lst})
data_df

In [ ]:
sns.set_style("ticks",{'axes.grid' : True})
fig, ax = plt.subplots(1, 1)
fig.set_size_inches(7, 4)

sns.lineplot(
    data=data_df,
    x="Segmentation", 
    y="NMI", 
    hue="Method", 
    style="Method",
    palette="rocket_r",
    markers=True, 
    dashes=False, 
    ax=ax,
)

ax.set_ylabel('Normalized Mutual Information (NMI)', fontsize=14)
ax.set_xlabel('Segmentation Solution', fontsize=14)
ax.set_xticks(np.arange(2, 33, 2))
ax.set_xticklabels(np.arange(2, 33, 2), fontsize=13)
ax.tick_params(axis='y', labelsize=13)
legend = ax.legend()
legend.set_title('')

fig.tight_layout()
fig.savefig(op.join(figures_dir, "Fig", "Fig-S5.png"), bbox_inches="tight", dpi=1000)
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

n_segments = 31
min_n_segments = 2
colors = ["#05BFDB", "#088395", "#0A4D68"]
min_peak, max_peak = gradient.min(), gradient.max()
boundaries = []
for seg_i, n_segment in enumerate(range(min_n_segments, n_segments + min_n_segments)):
    
    fig, ax = plt.subplots()
    fig.set_size_inches(10, 2)
    
    yticks = [0.5]
    for dict_i, results_dict in enumerate([kde_dict, kmeans_dict, pct_dict]):
        bound_arr = results_dict["boundaries"][seg_i]
        peaks_arr = results_dict["peaks"][seg_i]
        peaks_arr[0], peaks_arr[-1] = min_peak, max_peak

        x = []
        y = []
        x_err_i = []
        x_err_j = []
        for i in range(n_segment):
            bound_i, bound_j = bound_arr[i], bound_arr[i+1]
            peak = peaks_arr[i]
            x.append(peak)
            y.append(dict_i + 1)
            x_err_i.append(abs(peak - bound_i))
            x_err_j.append(abs(peak - bound_j))
    
        x_err = [x_err_i, x_err_j]
        (_, caps, _) = ax.errorbar(
            x, 
            y, 
            xerr=x_err, 
            fmt='o', 
            capsize=15, 
            elinewidth=3, 
            ecolor=colors[dict_i],
            markerfacecolor="r",
            markeredgecolor="r",
        )

        for cap in caps:
            cap.set_markeredgewidth(3)

        yticks.append(dict_i + 1)
    yticks.append(dict_i + 1.5)

    plt.yticks(yticks)
    ax.axis('off')
    fig.tight_layout()
    plt.savefig(op.join("../figures/Fig/segmentation", f"bound_{n_segment:02d}.eps"), bbox_inches="tight")
    
    plt.show()

In [ ]:
output_dir = "/Users/jperaza/Documents/GitHub/gradient-decoding/results/segmentation"
n_segments = 1
for method in ["PCT", "KMeans", "KDE"]:
    results_fn = op.join(output_dir, f"0_{method}_results.pkl")
    if not op.isfile(results_fn):
        if method == "PCT":
            # Percentile Segmentation
            print("\t\tRunning Percentile Segmentation...", flush=True)
            segment_method = PCTLSegmentation(results_fn, n_segments)
        elif method == "KMeans":
            # K-Means
            print("\t\tRunning K-Means Segmentation...", flush=True)
            segment_method = KMeansSegmentation(results_fn, n_segments)
        elif method == "KDE":
            # KDE
            print("\t\tRunning KDE Segmentation...", flush=True)
            segment_method = KDESegmentation(results_fn, n_segments)
        
        results_dict = segment_method.fit(gradient)

In [ ]:
output_dir = "/Users/jperaza/Documents/GitHub/gradient-decoding/results/segmentation"
for method in ["PCT", "KMeans", "KDE"]:
    new_results_fn = op.join(output_dir, f"0_{method}_results.pkl")
    old_results_fn = op.join(output_dir, f"{method}_results.pkl")
    results_fn = op.join(output_dir, f"new_{method}_results.pkl")

    with open(new_results_fn, "rb") as results_file:
        new_results_dict = pickle.load(results_file)
    
    with open(old_results_fn, "rb") as results_file:
        old_results_dict = pickle.load(results_file)
    
    results_dict = old_results_dict.copy()
    results_dict["segments"].insert(0, new_results_dict["segments"][0])
    results_dict["boundaries"].insert(0, new_results_dict["boundaries"][0])
    results_dict["peaks"].insert(0, new_results_dict["peaks"][0])
    results_dict["labels"].insert(0, new_results_dict["labels"][0])

    with open(results_fn, "wb") as segmentation_file:
        pickle.dump(results_dict, segmentation_file)

In [ ]:
import nibabel as nib

def scores_to_gii(scores, lh_fn, rh_fn):
    full_vertices = 64984
    hemi_vertices = full_vertices // 2

    grad_map_full = add_fslr_medial_wall(scores, split=False)
    grad_map_lh, grad_map_rh = (
        grad_map_full[:hemi_vertices],
        grad_map_full[hemi_vertices:],
    )
    grad_map_lh = np.float32(grad_map_lh)
    grad_map_rh = np.float32(grad_map_rh)

    grad_img_lh = GiftiImage()
    grad_img_rh = GiftiImage()
    grad_img_lh.add_gifti_data_array(GiftiDataArray(grad_map_lh))
    grad_img_rh.add_gifti_data_array(GiftiDataArray(grad_map_rh))

    # Write cortical gradient to Gifti files
    nib.save(grad_img_lh, lh_fn)
    nib.save(grad_img_rh, rh_fn)

In [ ]:
with open("/Users/jperaza/Documents/GitHub/gradient-decoding/results/segmentation/segments.pkl", "rb") as segmentation_file:
   grad_seg_dict = pickle.load(segmentation_file)

for method in ["PCT", "KMeans", "KDE"]:
    out_dir = f"/Users/jperaza/Documents/GitHub/gradient-decoding/results/segmentation/{method}_gradient-maps"
    grad_segments = grad_seg_dict[method]
    for grad_segment in grad_segments:
        seg_size = len(grad_segment)
        for map_i, grad_map in enumerate(grad_segment):
            print(seg_size, map_i)
            grad_map_lh_fn = op.join(
                out_dir,
                f"source-{method}{seg_size:02d}_desc-C{map_i:02d}_space-fsLR_den-32k_hemi-L_feature.func.gii",
            )
            grad_map_rh_fn = op.join(
                out_dir,
                f"source-{method}{seg_size:02d}_desc-C{map_i:02d}_space-fsLR_den-32k_hemi-R_feature.func.gii",
            )

            #if not (op.isfile(grad_map_lh_fn) and op.isfile(grad_map_rh_fn)):
            scores_to_gii(grad_map, grad_map_lh_fn, grad_map_rh_fn)

In [ ]:
for method in ["PCT", "KMeans", "KDE"]:
    out_dir = f"/Users/jperaza/Documents/GitHub/gradient-decoding/results/segmentation/{method}_confidence-maps"
    samples_arr_fn = op.join("../results", "segmentation", f"{method}_silhouette.npy")
    samples_arrays = np.load(samples_arr_fn)
    
    for map_i in range(samples_arrays.shape[0]):
        map_array = samples_arrays[map_i, :]
        seg_size = map_i + 2

        grad_map_lh_fn = op.join(
            out_dir,
            f"source-{method}{seg_size:02d}_desc-SilhouetteSamples_space-fsLR_den-32k_hemi-L_feature.func.gii",
        )
        grad_map_rh_fn = op.join(
            out_dir,
            f"source-{method}{seg_size:02d}_desc-SilhouetteSamples_space-fsLR_den-32k_hemi-R_feature.func.gii",
        )

        scores_to_gii(map_array, grad_map_lh_fn, grad_map_rh_fn)